In [76]:
import pandas as pd; import numpy as np; import os as os; import multiprocessing as mp
import matplotlib.pyplot as plt; import seaborn as sns; import scipy.interpolate as sp
import statsmodels.formula.api as smf

In [77]:
if os.name=="nt":
    path ="J:\\Project\\Cost_Effectiveness\\dev\\"
else:
    path = "\home\\j\\Project\Cost_Effectiveness\\dev\\"
    
ltable_path ="data\\gbd"
all_data = "data_processed"

### Load in Data

In [78]:
all_death_df = pd.read_csv(path+'\\data_processed\\Mortality_Rates.csv')
ihd_death_df = pd.read_csv(path +'\\data_processed\\ihd_mortality_rate.csv')
ihd_incidence_df = pd.read_csv(path + '\\data_processed\\IHD incidence rates.csv')
cohort = pd.read_csv(path+all_data+"\\level2_population.csv")
cohort.rename(columns={'age':'Age', 'year_id':'Year', 'IHD':'IHD incidence'}, inplace=True) ## this needs to change
ltable = pd.read_csv(path+ltable_path+ "\\interpolated_reference_life_table.csv")

## Define an interpolator


In [79]:
### interpolator
def gen_curve(df, df_id='Mortality_Rate', time =24, inv=0):
    df['inv_prob'] =  np.exp(-df[df_id])
    df.Year -= np.min(df.Year)
    df.Age -= df.Year 
    df = df[df.Age>0]
    df = df[df.Age<=80]
    df['cumprod'] = df.groupby(['Age', 'sex']).inv_prob.agg(np.cumprod)
    
    df.Year+=1
    # could not control when random num was out of bounds high or low
    ## function defined below takes twice the time now... not worrying about that
    for i in np.arange(2):
        for s in np.unique(df.sex):
                temp = pd.DataFrame(columns=df.columns)
                temp['Age'] =np.unique(df.Age)
                temp['sex']= s
                temp['Year']=np.where(i==1, 0, time)
                temp['cumprod'] = i
                df = df.append(temp)
            
    model= sp.LinearNDInterpolator(df[['cumprod','Age', 'sex']], df['Year'])
    ## I need to backcalc time didn't know of a better way.
    inv_model = sp.LinearNDInterpolator(df[['Year','Age', 'sex']],df['cumprod'])
    
    def func(self, ind, rand_seq, name):
               
        df =self.attributes.ix[ind].copy()
        # convert into rate
        rand_seq = -np.log(1-(1-rand_seq))
        rand_seq += self.rate_adj[name][ind,0].reshape(len(ind[0],))## check the ordering
        rand_seq*= self.rate_adj[name][ind,1].reshape(len(ind[0],))
        rand_seq = np.exp(-rand_seq)
        df['cumprod'] =rand_seq.T
        fit =model(df[['cumprod','Age', 'sex']])
        fit = np.round(fit,3)
        return fit
    
    def invfunc(self,ind, name):
        df =self.attributes.ix[ind].copy()
        time = self.run_time[ind]
        df['Year'] =time
        fit =np.array(inv_model(df[['Year','Age', 'sex']]))
        fit = -np.log(1-(1-fit))
        fit /= self.rate_adj[name][ind,1].reshape(len(ind[0]),)
        fit -= self.rate_adj[name][ind,0].reshape(len(ind[0]),)
        fit = np.exp(-fit)
        fit = np.round(fit,3)
        return fit
    
           
    return {'func':func, 'invfunc': invfunc}

### Define sim

In [80]:
class sim(object):
    '''TODO: add documentation
    FIX: IHD dead indicator'''    
    def __init__(self,n=2e4, time =24, seed=1):
        np.random.seed(seed)
        self.n = int(n)
        self.time = int(time)
        self.attributes = pd.DataFrame()
        
        
        ## define stuff that will be used later on
        self.events= {}
        self.event_names =[]
        self.updates = {}
        self.deadly_events = []

        self.run_time = np.repeat(0, n)
        self.ledger = pd.DataFrame(columns=['simulant_id', 'event_name', 'event_time'])
        self.rate_adj = {}
        self.at_risk = pd.DataFrame()
        self.temp_log =[]
     
    def add_attribute(self,colnames, data):
        self.attributes[colnames]=data
    def add_events(self,name, funcs_dict, deadly, init_at_risk ):
        self.event_names.append(name)
        self.events.update({name: funcs_dict})
        if deadly==1:
            self.deadly_events.append(name)
        self.rate_adj.update({name: np.vstack((np.zeros(self.n) ,np.ones(self.n))).T})
        self.at_risk[name] = init_at_risk
        
    def add_updates(self,name, funcs_dict):
        self.updates.update({name:funcs_dict})
    
    
    def run(self):
        #set indicator to index attributes
        ind = np.where(self.attributes.Dead==0)
        
        ## run till I kill everyone.
        while np.sum(self.attributes.Dead==0):
            temp_log = np.empty([len(ind[0]), len(self.events)])
            rand = np.random.uniform(0,1, [len(ind), len(self.events)])
            
            ## loop thru events
            for i in np.arange(len(self.events)):
                name = self.event_names[i]
                ## need to find out where time t occurs on the [0,1] range and scale new random numbers
                ## this is by far the slowest and there are other ways this can be done 
                time_scale = self.events[name]['invfunc'](self, ind, name )*rand[:,i]
                ## get time of event
                temp_log[:,i] = self.at_risk[name].ix[ind]*999+self.events[name]['func'](self, ind, time_scale, name)
            
            self.temp_log.append(temp_log)
            ## get soonest event
            min_ind =temp_log.argmin(axis=1) # get templog index
            min_time = temp_log.min(axis=1) # get time it occured
            min_name = np.array([self.event_names[i] for i in min_ind]) ## get min event name
            
            ## ensure general mortality trumps all else
            min_ind = np.where(min_time==temp_log[:,0], 0, min_ind) ## requires mortality to be added first.. may change
           
            ## update ledger
            self.ledger = self.ledger.append(pd.DataFrame({'simulant_id':self.attributes.ix[ind]['simulant_id'].values,
                            'event_name': min_name,
                            'event_time':min_time}))
            # update run time
            self.run_time[ind] = min_time.copy()
            ## update attributes based on events that occured
            for i in np.arange(len(self.updates)):
                name = self.updates.keys()[i]
                self.updates[name](self, ind, min_ind, min_name)
            
            ## kill people off
            self.attributes['Dead'].ix[ind] =np.where(np.in1d(min_name, self.deadly_events), 1, 0)
            ind =np.where(self.attributes.Dead==0)

# Level two
#### steps:
1. define attributes
2. define events
3. define updates
4. run model
4. analyze results

In [81]:
###################
#add attributes
model = sim()
model.add_attribute(cohort.columns,cohort)
model.add_attribute('IHD Dead', 0) ## incase we want to seperate by death
model.add_attribute('tx', 0)


In [82]:
######################################
#add time-to-event events

## the loop may be unnecessary
event_col= ['Mortality_Rate', 'Mortality_Rate', 'Incidence']
event_data = [all_death_df, ihd_death_df, ihd_incidence_df]
event_names = ['Dead', 'IHD Dead', 'IHD incidence']
dead_events = [ 0, 1, 0]
ls_at_risk = [cohort.Alive, cohort['IHD incidence'], np.where(cohort['IHD incidence']==1, 0,1)]
## when all said and done... not sure this loop is worthwhile. There may be a better way.
for i in np.arange(len(event_col)):
      
    name = event_names[i]
    model.add_events(name, gen_curve(event_data[i], event_col[i]), dead_events[i],ls_at_risk[i])
    
##################################
# add tx

#define tx event
def txfunc(self, ind, rand_seq, name):
    df = self.attributes.ix[ind].copy()
    tx_ind = np.where((df.Age>=20) & (df.tx==0) & (5>= self.run_time[ind]), 5, 999) ## no tx untill 1990 for >25
    tx_ind = np.where((df.tx==0) & (df.Age<20) & (25>=df.Age+self.run_time[ind]), 25-df.Age, tx_ind)
    return tx_ind
def txinvfunc(self,ind, name ):
    return 1
## add tx
model.add_events('tx', {'func':txfunc, 'invfunc': txinvfunc},0,0)

In [83]:
#######################################
## add updates

# define updates
def ihd_update(self, ind, min_ind, min_name):
    self.attributes['IHD incidence'].ix[ind]= np.where(min_name=='IHD incidence'  , 1, self.attributes['IHD incidence'].ix[ind])
    
    self.at_risk['IHD Dead'].ix[ind] = np.where(min_name=='IHD incidence' , 0,self.at_risk['IHD Dead'].ix[ind] )
    self.at_risk['IHD incidence'].ix[ind] = np.where(min_name=='IHD incidence', 1, self.at_risk['IHD incidence'].ix[ind])

def tx_update(self, ind, min_ind, min_name):
    
    self.attributes['tx'].ix[ind] = np.where(min_name=='tx' , 1, self.attributes['tx'].ix[ind])
    self.at_risk['tx'].ix[ind] = np.where(min_name=='tx', 1, self.at_risk['tx'].ix[ind])
    self.rate_adj['IHD incidence'][ind,1] *=np.where(min_name=='tx', 2, 1)
    
    
model.add_updates('ihd_update', ihd_update)
model.add_updates('tx_update', tx_update)

In [84]:
model.run()

In [85]:
model.ledger

,event_name,event_time,simulant_id
0,tx,20.0,0.0
1,tx,18.0,1.0
2,tx,5.0,2.0
3,tx,11.0,3.0
4,tx,5.0,4.0
5,tx,5.0,5.0
6,tx,8.0,6.0
7,tx,5.0,7.0
8,tx,15.0,8.0
9,tx,6.0,9.0
